**Import Libraries**

In [2]:
#Import Libraries
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Load the dataset**

In [4]:
input_path = '/content/drive/MyDrive/NM/tea sickness dataset/Anthracnose'
output_path = '/content/drive/MyDrive/NM/tea sickness dataset'

**Preprocessing**

In [5]:
imageSize=[224,224]

In [6]:
vgg = VGG16(input_shape=imageSize +[3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 4s 0us/step


In [7]:
#don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [8]:
#our layers - you can add more if you want
x = Flatten()(vgg.output)

**Adding Layers**

In [9]:
1 #Our layers - you can add more if you want
2 #Adding Flatten Layers
x = Flatten()(vgg.output)

In [10]:
#Adding output layer
prediction = Dense(8, activation='softmax')(x)

**Defining the model**

In [11]:
#Create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
#tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], run_eagerly=True
)

In [14]:
#import the image datagenerator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
trainPath = '/content/drive/MyDrive/tea sickness dataset'

training_set = train_datagen.flow_from_directory(trainPath,
                                              target_size = (224,224),
                                              batch_size = 32,
                                              class_mode = 'categorical')

testPath = '/content/drive/MyDrive/tea sickness dataset'

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 885 images belonging to 8 classes.
Found 885 images belonging to 8 classes.


In [17]:
training_set.class_indices

{'Anthracnose': 0,
 'algal leaf': 1,
 'bird eye spot': 2,
 'brown blight': 3,
 'gray light': 4,
 'healthy': 5,
 'red leaf spot': 6,
 'white spot': 7}

In [18]:
# fit the model
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

<ipython-input-18-15eb5a7e606d>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/30
28/28 [==============================] - 590s 20s/step - loss: 1.6613 - accuracy: 0.4362 - val_loss: 0.8899 - val_accuracy: 0.6678
Epoch 2/30
28/28 [==============================] - 169s 6s/step - loss: 0.7903 - accuracy: 0.7130 - val_loss: 0.5461 - val_accuracy: 0.7977
Epoch 3/30
28/28 [==============================] - 172s 6s/step - loss: 0.5673 - accuracy: 0.8102 - val_loss: 0.4129 - val_accuracy: 0.8655
Epoch 4/30
28/28 [==============================] - 165s 6s/step - loss: 0.4432 - accuracy: 0.8565 - val_loss: 0.3072 - val_accuracy: 0.9175
Epoch 5/30
28/28 [==============================] - 167s 6s/step - loss: 0.3732 - accuracy: 0.8949 - val_loss: 0.2512 - val_accuracy: 0.9345
Epoch 6/30
28/28 [==============================] - 169s 6s/step - loss: 0.3813 - accuracy: 0.8701 - val_loss: 0.2263 - val_accuracy: 0.9446
Epoch 7/30
28/28 [==============================] - 167s 6s/step - loss: 0.3506 - accuracy: 0.8746 - val_loss: 0.2479 - val_accuracy: 0.9254
Epoch 8/30
2

**Save the Model**

In [19]:
#save the model
model.save('vgg-16-Tea-leaves-disease-model.h5')

**Test the model**

In [20]:
img=image.load_img(r'/content/drive/MyDrive/tea sickness dataset/brown blight/UNADJUSTEDNONRAW_thumb_110.jpg',target_size=(224,224))

#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
       'algal leaf',
       'bird eye spot',
       'brown blight',
       'gray blight',
       'healthy',
       'red leaf spot',
       'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 1s 607ms/step


'algal leaf'

In [21]:
from google.colab import files

# Replace 'model.h5' with the name of your .h5 file
files.download('vgg-16-Tea-leaves-disease-model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>